In [13]:
# pytorch
import torch
import torchvision
from torchvision import transforms, datasets, models
from torchsummary import summary
from torchvision import transforms
from PIL import Image
from torch.utils.data import Dataset, DataLoader

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# other
import numpy as np
import matplotlib.pyplot as plt
import copy
import time
import glob

In [14]:
class appleDataset(Dataset):
    def __init__(self, path, train, transform=None):
        self.path = path
        if train:
            self.AbNorm_path = path + '/train/AbNorm/'
            self.Mot_path = path + '/train/Mot/'
            self.Norm_path = path + '/train/Norm/'
        else:
            self.AbNorm_path = path + '/test/AbNorm/'
            self.Mot_path = path + '/test/Mot/'
            self.Norm_path = path + '/test/Norm/'
        
        self.AbNorm_img_list = glob.glob(self.AbNorm_path + '/*.jpg')
        self.Mot_img_list = glob.glob(self.Mot_path + '/*.jpg')
        self.Norm_img_list = glob.glob(self.Norm_path + '/*.jpg')

        self.transform = transform

        self.img_list = self.AbNorm_img_list + self.Mot_img_list + self.Norm_img_list
        self.class_list = [0] * len(self.AbNorm_img_list) + [1] * len(self.Mot_img_list) + [2] * len(self.Norm_img_list)
        
    def __len__(self):
        return len(self.img_list)
    
    def __getitem__(self, idx):
        img_path = self.img_list[idx]
        label = self.class_list[idx]
        img = Image.open(img_path)

        if self.transform is not None:
            img = self.transform(img)

        return img, label


In [15]:
transform = transforms.Compose([transforms.ToTensor()])

train_dataset = appleDataset(path='./input', train=True, transform=transform)
trainloader = DataLoader(dataset=train_dataset,
                        batch_size=5,
                        shuffle=True,
                        drop_last=False)
    
test_dataset = appleDataset(path='./input', train=False, transform=transform)
testloader = DataLoader(dataset=test_dataset,
                        batch_size=5,
                        shuffle=True,
                        drop_last=False
                        )

In [ ]:
for epoch in range(2):
    print(f"epoch : {epoch} ")
    for batch in testloader:
        img, label = batch
        print(img.size(), label)

In [6]:
classes = ('Norm','Mot','AbNorm')


In [17]:
resnet_pt = models.resnet18(weights=True)
# freezing
for param in resnet_pt.parameters():
    param.requires_grad = False

# fc layer 수정
fc_in_features = resnet_pt.fc.in_features
resnet_pt.fc = nn.Linear(fc_in_features, len(classes))
resnet_pt = resnet_pt.to(device)

In [18]:
summary(resnet_pt, (3,32,32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 16, 16]           9,408
       BatchNorm2d-2           [-1, 64, 16, 16]             128
              ReLU-3           [-1, 64, 16, 16]               0
         MaxPool2d-4             [-1, 64, 8, 8]               0
            Conv2d-5             [-1, 64, 8, 8]          36,864
       BatchNorm2d-6             [-1, 64, 8, 8]             128
              ReLU-7             [-1, 64, 8, 8]               0
            Conv2d-8             [-1, 64, 8, 8]          36,864
       BatchNorm2d-9             [-1, 64, 8, 8]             128
             ReLU-10             [-1, 64, 8, 8]               0
       BasicBlock-11             [-1, 64, 8, 8]               0
           Conv2d-12             [-1, 64, 8, 8]          36,864
      BatchNorm2d-13             [-1, 64, 8, 8]             128
             ReLU-14             [-1, 6

In [19]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet_pt.parameters(), lr=0.001,
                      momentum=0.9)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

In [20]:
# Training
def train(epoch, model, criterion, optimizer):
    model.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, labels) in enumerate(trainloader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()*inputs.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    
    epoch_loss = train_loss/total
    epoch_acc = correct/total*100
    print("Train | Loss:%.4f Acc: %.2f%% (%s/%s)" 
        % (epoch_loss, epoch_acc, correct, total))
    return epoch_loss, epoch_acc

def test(epoch, model, criterion, optimizer):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, labels) in enumerate(testloader):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            test_loss += loss.item()*inputs.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
        
        epoch_loss = test_loss/total
        epoch_acc = correct/total*100
        print("Test | Loss:%.4f Acc: %.2f%% (%s/%s)" 
            % (epoch_loss, epoch_acc, correct, total))
    return epoch_loss, epoch_acc

In [37]:
tart_time = time.time()
best_acc = 0
epoch_length = 40
save_loss = {"train":[],
             "test":[]}
save_acc = {"train":[],
             "test":[]}
for epoch in range(epoch_length):
    print("Epoch %s" % epoch)
    train_loss, train_acc = train(epoch, resnet_pt, criterion, optimizer)
    save_loss['train'].append(train_loss)
    save_acc['train'].append(train_acc)

    test_loss, test_acc = test(epoch, resnet_pt, criterion, optimizer)
    save_loss['test'].append(test_loss)
    save_acc['test'].append(test_acc)

    scheduler.step()

    # Save model
    if test_acc > best_acc:
        best_acc = test_acc
        best_model_wts = copy.deepcopy(resnet_pt.state_dict())
    resnet_pt.load_state_dict(best_model_wts)

learning_time = time.time() - start_time
print(f'**Learning time: {learning_time // 60:.0f}m {learning_time % 60:.0f}s')

Epoch 0
Train | Loss:0.2902 Acc: 89.13% (37816/42426)
Test | Loss:0.1340 Acc: 95.25% (5054/5306)
Epoch 1
Train | Loss:0.2594 Acc: 90.55% (38415/42426)
Test | Loss:0.1799 Acc: 93.22% (4946/5306)
Epoch 2
Train | Loss:0.2542 Acc: 90.79% (38519/42426)
Test | Loss:0.1201 Acc: 95.67% (5076/5306)
Epoch 3
Train | Loss:0.2531 Acc: 90.96% (38592/42426)
Test | Loss:0.1844 Acc: 93.12% (4941/5306)
Epoch 4
Train | Loss:0.2468 Acc: 91.09% (38644/42426)
Test | Loss:0.1077 Acc: 96.14% (5101/5306)
Epoch 5
Train | Loss:0.2568 Acc: 90.82% (38532/42426)
Test | Loss:0.1240 Acc: 95.65% (5075/5306)
Epoch 6
Train | Loss:0.2419 Acc: 91.34% (38750/42426)
Test | Loss:0.1068 Acc: 96.06% (5097/5306)
Epoch 7
Train | Loss:0.2411 Acc: 91.14% (38665/42426)
Test | Loss:0.1248 Acc: 95.27% (5055/5306)
Epoch 8
Train | Loss:0.2495 Acc: 91.15% (38671/42426)
Test | Loss:0.1171 Acc: 95.67% (5076/5306)
Epoch 9
Train | Loss:0.2576 Acc: 90.64% (38453/42426)
Test | Loss:0.1349 Acc: 94.84% (5032/5306)
Epoch 10
Train | Loss:0.2475 A

KeyboardInterrupt: 